In [5]:
from __future__ import absolute_import, division, print_function

import os

import tensorflow as tf
from tensorflow import keras

tf.__version__

'1.8.0'

In [6]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

In [7]:
# Returns a short sequential model
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation=tf.nn.relu, input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10, activation=tf.nn.softmax)
  ])
  
  model.compile(optimizer=tf.keras.optimizers.Adam(), 
                loss=tf.keras.losses.sparse_categorical_crossentropy,
                metrics=['accuracy'])
  
  return model


# Create a basic model instance
model = create_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 512)               401920    
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [10]:
checkpoint_path = "/home/yqw/Github/test1/tensorflow_test/1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, 
                                                 save_weights_only=True,
                                                 verbose=1)

model = create_model()

model.fit(train_images, train_labels,  epochs = 10, 
          validation_data = (test_images,test_labels),
          callbacks = [cp_callback])  # pass callback to training


Train on 1000 samples, validate on 1000 samples
Epoch 1/10
1000/1000 [==============================] - 0s 266us/step - loss: 1.1563 - acc: 0.6750 - val_loss: 0.7066 - val_acc: 0.7800

Epoch 00001: saving model to /home/yqw/Github/test1/tensorflow_test/1/cp.ckpt
Epoch 2/10
1000/1000 [==============================] - 0s 117us/step - loss: 0.4345 - acc: 0.8880 - val_loss: 0.5207 - val_acc: 0.8440

Epoch 00002: saving model to /home/yqw/Github/test1/tensorflow_test/1/cp.ckpt
Epoch 3/10
1000/1000 [==============================] - 0s 117us/step - loss: 0.2817 - acc: 0.9280 - val_loss: 0.4570 - val_acc: 0.8510

Epoch 00003: saving model to /home/yqw/Github/test1/tensorflow_test/1/cp.ckpt
Epoch 4/10
1000/1000 [==============================] - 0s 112us/step - loss: 0.2090 - acc: 0.9540 - val_loss: 0.4372 - val_acc: 0.8590

Epoch 00004: saving model to /home/yqw/Github/test1/tensorflow_test/1/cp.ckpt
Epoch 5/10
1000/1000 [==============================] - 0s 113us/step - loss: 0.1523 - acc: 

In [11]:
!ls {checkpoint_dir}

cp.ckpt


In [12]:
ls {checkpoint_dir}

cp.ckpt


In [13]:
model = create_model()

loss, acc = model.evaluate(test_images, test_labels)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 103us/step
Untrained model, accuracy: 10.90%


In [14]:
model.load_weights(checkpoint_path)
loss,acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 23us/step
Restored model, accuracy: 87.10%


In [15]:
# include the epoch in the file name. (uses `str.format`)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, verbose=1, save_weights_only=True,
    # Save weights, every 5-epochs.
    period=5)

model = create_model()
model.fit(train_images, train_labels,
          epochs = 50, callbacks = [cp_callback],
          validation_data = (test_images,test_labels),
          verbose=0)


Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


In [18]:
# Create some variables.
v1 = tf.get_variable("v1", shape=[3], initializer = tf.zeros_initializer)
v2 = tf.get_variable("v2", shape=[5], initializer = tf.zeros_initializer)

inc_v1 = v1.assign(v1+1)
dec_v2 = v2.assign(v2-1)

# Add an op to initialize the variables.
init_op = tf.global_variables_initializer()

# Add ops to save and restore all the variables.
saver = tf.train.Saver()

# Later, launch the model, initialize the variables, do some work, and save the
# variables to disk.
with tf.Session() as sess:
  sess.run(init_op)
  # Do some work with the model.
  inc_v1.op.run()
  dec_v2.op.run()
  # Save the variables to disk.
  save_path = saver.save(sess, "/tmp/model.ckpt")
  print("Model saved in path: %s" % save_path)

Model saved in path: /tmp/model.ckpt


In [19]:
tf.reset_default_graph()

# Create some variables.
v1 = tf.get_variable("v1", shape=[3])
v2 = tf.get_variable("v2", shape=[5])

# Add ops to save and restore all the variables.
saver = tf.train.Saver()

# Later, launch the model, use the saver to restore variables from disk, and
# do some work with the model.
with tf.Session() as sess:
  # Restore variables from disk.
  saver.restore(sess, "/tmp/model.ckpt")
  print("Model restored.")
  # Check the values of the variables
  print("v1 : %s" % v1.eval())
  print("v2 : %s" % v2.eval())

INFO:tensorflow:Restoring parameters from /tmp/model.ckpt
Model restored.
v1 : [1. 1. 1.]
v2 : [-1. -1. -1. -1. -1.]


In [20]:
tf.reset_default_graph()
# Create some variables.
v1 = tf.get_variable("v1", [3], initializer = tf.zeros_initializer)
v2 = tf.get_variable("v2", [5], initializer = tf.zeros_initializer)

# Add ops to save and restore only `v2` using the name "v2"
saver = tf.train.Saver({"v2": v2})

# Use the saver object normally after that.
with tf.Session() as sess:
  # Initialize v1 since the saver will not.
  v1.initializer.run()
  saver.restore(sess, "/tmp/model.ckpt")

  print("v1 : %s" % v1.eval())
  print("v2 : %s" % v2.eval())

INFO:tensorflow:Restoring parameters from /tmp/model.ckpt
v1 : [0. 0. 0.]
v2 : [-1. -1. -1. -1. -1.]


In [21]:
# import the inspect_checkpoint library
from tensorflow.python.tools import inspect_checkpoint as chkp

# print all tensors in checkpoint file
chkp.print_tensors_in_checkpoint_file("/tmp/model.ckpt", tensor_name='', all_tensors=True)

# tensor_name:  v1
# [ 1.  1.  1.]
# tensor_name:  v2
# [-1. -1. -1. -1. -1.]

# print only tensor v1 in checkpoint file
chkp.print_tensors_in_checkpoint_file("/tmp/model.ckpt", tensor_name='v1', all_tensors=False)

# tensor_name:  v1
# [ 1.  1.  1.]

# print only tensor v2 in checkpoint file
chkp.print_tensors_in_checkpoint_file("/tmp/model.ckpt", tensor_name='v2', all_tensors=False)

# tensor_name:  v2
# [-1. -1. -1. -1. -1.]

tensor_name:  Adam/beta_1
0.9
tensor_name:  Adam/beta_2
0.999
tensor_name:  Adam/decay
0.0
tensor_name:  Adam/iterations
0
tensor_name:  Adam/lr
0.001
tensor_name:  Adam_1/beta_1
0.9
tensor_name:  Adam_1/beta_2
0.999
tensor_name:  Adam_1/decay
0.0
tensor_name:  Adam_1/iterations
0
tensor_name:  Adam_1/lr
0.001
tensor_name:  Adam_2/beta_1
0.9
tensor_name:  Adam_2/beta_2
0.999
tensor_name:  Adam_2/decay
0.0
tensor_name:  Adam_2/iterations
0
tensor_name:  Adam_2/lr
0.001
tensor_name:  Adam_3/beta_1
0.9
tensor_name:  Adam_3/beta_2
0.999
tensor_name:  Adam_3/decay
0.0
tensor_name:  Adam_3/iterations
0
tensor_name:  Adam_3/lr
0.001
tensor_name:  Adam_4/beta_1
0.9
tensor_name:  Adam_4/beta_2
0.999
tensor_name:  Adam_4/decay
0.0
tensor_name:  Adam_4/iterations
0
tensor_name:  Adam_4/lr
0.001
tensor_name:  Adam_5/beta_1
0.9
tensor_name:  Adam_5/beta_2
0.999
tensor_name:  Adam_5/decay
0.0
tensor_name:  Adam_5/iterations
0
tensor_name:  Adam_5/lr
0.001
tensor_name:  Adam_6/beta_1
0.9
tensor_name:

In [22]:
print(tf.add(1, 2))
print(tf.add([1, 2], [3, 4]))
print(tf.square(5))
print(tf.reduce_sum([1, 2, 3]))
print(tf.encode_base64("hello world"))

# Operator overloading is also supported
print(tf.square(2) + tf.square(3))

Tensor("Add:0", shape=(), dtype=int32)
Tensor("Add_1:0", shape=(2,), dtype=int32)
Tensor("Square:0", shape=(), dtype=int32)
Tensor("Sum:0", shape=(), dtype=int32)
Tensor("EncodeBase64:0", shape=(), dtype=string)
Tensor("add:0", shape=(), dtype=int32)
